In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier

In [61]:
train_df = pd.read_csv('data/train_catugra.csv', sep=',')
val_df = pd.read_csv('data/test_catugra.csv', sep=',')

train_df

,Unnamed: 0,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,...,children_subject,rural_subject,services_subject,communication_subject,district_population,mean_income_district,children_district,rural_district,services_district,communication_district
0,0,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00101,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
1,1,1,2020-05-01,Город,Москва,Москва,8611aa01fffffff,55.975851,37.237085,0.00000,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
2,2,1,2020-05-01,Город,Москва,Москва,861181b6fffffff,55.622721,37.695121,0.00339,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
3,3,1,2020-05-01,Город,Москва,Москва,8611aa017ffffff,55.941586,37.157487,0.00048,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
4,4,1,2020-05-01,Город,Москва,Москва,8611aa637ffffff,55.797494,37.676200,0.00164,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,294253,0,2020-12-01,Область,Магаданская,Ола,8617154afffffff,59.577462,151.294079,0.00045,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7
294254,294254,0,2020-12-01,Область,Магаданская,Ягодное,861735767ffffff,62.514463,149.591693,0.00048,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7
294255,294255,0,2020-12-01,Область,Магаданская,Магадан,8617142cfffffff,59.598694,150.810630,NaN,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7
294256,294256,0,2020-12-01,Автономный Округ,Чукотский,Певек,8604ebc5fffffff,69.703205,170.185894,0.00001,...,22.2,28.8,26.8,4.6,8124.0,39822.4,20.8,27.0,25.5,3.7


In [56]:
val_df

,Unnamed: 0,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,...,children_subject,rural_subject,services_subject,communication_subject,district_population,mean_income_district,children_district,rural_district,services_district,communication_district
0,0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
1,1,1,2020-11-01,Город,Москва,Москва,8611aa70fffffff,55.879910,37.583383,0.00265,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
2,2,1,2020-11-01,Город,Москва,Москва,8611aa627ffffff,55.855726,37.669858,0.00046,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
3,3,1,2020-11-01,Город,Москва,Москва,861181b2fffffff,55.496542,37.542584,0.00304,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
4,4,1,2020-11-01,Город,Москва,Москва,8611aa4cfffffff,55.613025,37.529612,0.00183,...,15.6,1.6,34.1,3.3,39251.0,46880.0,16.6,17.6,28.7,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,152761,0,2021-06-01,Область,Магаданская,Магадан,8617142efffffff,59.652400,150.759792,0.00024,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7
152762,152762,0,2021-06-01,Область,Магаданская,Магадан,861710967ffffff,59.544997,150.861279,0.00044,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7
152763,152763,0,2021-06-01,Область,Магаданская,Магадан,861710967ffffff,59.544997,150.861279,0.00023,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7
152764,152764,0,2021-06-01,Область,Магаданская,Магадан,861710967ffffff,59.544997,150.861279,0.00022,...,18.6,3.9,30.6,4.1,8124.0,39822.4,20.8,27.0,25.5,3.7


In [57]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294258 entries, 0 to 294257
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              294258 non-null  int64  
 1   label                   294258 non-null  int64  
 2   period                  294258 non-null  object 
 3   subject_type            294258 non-null  object 
 4   subject_name            294258 non-null  object 
 5   city_name               294247 non-null  object 
 6   hex                     294258 non-null  object 
 7   hex_lat                 294258 non-null  float64
 8   hex_lon                 294258 non-null  float64
 9   f1                      258144 non-null  float64
 10  f2                      262930 non-null  float64
 11  f3                      265786 non-null  float64
 12  f4                      279564 non-null  float64
 13  f5                      279564 non-null  float64
 14  f6                  

## Random modeling

In [59]:
N = 5
top5p = int(val_df.shape[0] * 0.05)
random_res = []

for i in range(N):
    preds = val_df.label.sample(val_df.shape[0]).reset_index(drop=True)
    val_df['pred'] = preds
    random_res.append(val_df.sort_values('pred', ascending=False).iloc[:top5p].label.sum()/top5p)

np.mean(random_res), random_res

(0.02710133542812255,
 [0.030243519245875882,
  0.012045037968054464,
  0.02958889761717727,
  0.03233830845771144,
  0.03129091385179366])

In [44]:
val_df.label.sample(val_df.shape[0]).iloc[:top5p].sum()

152

## Response modeling

In [62]:
cat_cols = train_df.select_dtypes(include=['object']).columns.values
train_df[cat_cols] = train_df[cat_cols].fillna('NaN')
val_df[cat_cols] = val_df[cat_cols].fillna('NaN')


ctb = CatBoostClassifier(verbose=200, task_type='GPU')
ctb.fit(train_df.drop('label', axis=1), train_df.label, cat_features=cat_cols)


Learning rate set to 0.02466
0:	learn: 0.6450974	total: 87.1ms	remaining: 1m 27s
200:	learn: 0.0762588	total: 34.5s	remaining: 2m 17s
400:	learn: 0.0689701	total: 50.6s	remaining: 1m 15s
600:	learn: 0.0660056	total: 1m 12s	remaining: 48.3s
800:	learn: 0.0635397	total: 1m 34s	remaining: 23.4s
999:	learn: 0.0617716	total: 2m 1s	remaining: 0us


In [64]:
val_df = pd.read_csv('data/test_catugra.csv', sep=',')
val_df[cat_cols] = val_df[cat_cols].fillna('NaN')

preds = ctb.predict_proba(val_df.drop('label', axis=1))[:,1]
val_df['pred'] = preds

In [65]:
top5p = int(val_df.shape[0] * 0.05)
res_response = val_df.sort_values('pred', ascending=False).iloc[:top5p].label.sum()/top5p
res_response

0.10971458496988741